In [1]:
# standard library
import _pickle as cPickle
import gzip
# third party libraries
import numpy as np

<h3>load_data()</h3> 

Returns the MNIST data as a tuple containing the training data, the validation data, and the test data.

 The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

 The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

 The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

 This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    

In [ ]:
def load_data():
    f = gzip.open('mnist.pkl.gz','rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding="latin1")
    print(training_data)
    f.close()
    return (training_data, validation_data, test_data)


<h3>load_data_wrapper()</h3>
 
 Returns a tuple containing (``training_data``, ``validation_data``,
    ``test_data``). Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

  In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

  ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

  Obviously, this means we're using slightly different formats for
    the training data and the validation / test data. 

In [2]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [3]:
training_data, validation_data, test_data = \
... load_data_wrapper()

(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([5, 0, 4, ..., 8, 4, 8], dtype=int64))


In [4]:
print(len(training_data))

50000


<h2 align="center">Model</h2>

A module to implement the ``stochastic gradient descent`` learning
algorithm for a ``feedforward`` neural network.  Gradients are calculated
using ``backpropagation``.  

``__init__(sizes)`` : The list **_sizes_** contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers.
        
``feed forward(a)`` : Returns the output of the network if ``a`` is input.

``SGD(training_data, epochs, mini_batch_size, eta, test_data)`` : Trains the neural network using mini-batch stochastic
        gradient descent.  The _**training_data**_ is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If _**test_data**_ is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially.

``update_mini_batch( mini_batch, eta)`` : Updates the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The _**mini_batch**_ is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate.
        
``backprop(x, y)`` : Returns a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function **C_x**.  **_grad_b_** and
        **_grad_w_** are layer-by-layer lists of numpy arrays, similar
        to _**self.biases**_ and _**self.weights**_.
        
``evaluate(test_data)`` : Returns the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation.
        
``cost_derivative(output_activations, y)`` : Return the vector of partial derivatives  _**dC_x/da**_ for the output activations.
        
 ``sigmoid(z) and sigmoid_prime(z)`` : The sigmoid function and its derivative respectively.

In [5]:
import random
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])]
    
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
            
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data = None):
        if test_data : 
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0,n,mini_batch_size)
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                res = self.evaluate(test_data)
                print( "Epoch {0}: {1} / {2} , accuracy : {3} %".format(
                    j, res, n_test,(res/n_test*100)))
            else:
                print( "Epoch {0} complete".format(j))
        
    
    def update_mini_batch(self, mini_batch, eta):
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            grad_b = [gb+dgb for gb, dgb in zip(grad_b, delta_grad_b)]
            grad_w = [gw+dgw for gw, dgw in zip(grad_w, delta_grad_w)]
        self.weights = [w-(eta/len(mini_batch))*gw
                        for w, gw in zip(self.weights, grad_w)]
        self.biases = [b-(eta/len(mini_batch))*gb
                       for b, gb in zip(self.biases, grad_b)]
        
        
    def backprop(self, x, y):
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        #feedforward
        activation = x
        activations = [x] #list to store all activations layer by layer
        zs = [] #list to store all the z vectors, layer by layer
        for b,w in zip(self.biases, self.weights):
            z = np.dot(w,activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        #backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (grad_b, grad_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
    
 #end of class Network
    
### misc. functions   
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [6]:
net = Network([784, 50, 10])

In [7]:
#  30 epochs, with a mini-batch size of 10, 
# and a learning rate of  η=3.0

net.SGD(training_data, 30, 20, 3, test_data=test_data)

Epoch 0: 7249 / 10000 , accuracy : 72.49 %
Epoch 1: 9090 / 10000 , accuracy : 90.9 %
Epoch 2: 9289 / 10000 , accuracy : 92.89 %
Epoch 3: 9351 / 10000 , accuracy : 93.51 %
Epoch 4: 9395 / 10000 , accuracy : 93.95 %
Epoch 5: 9406 / 10000 , accuracy : 94.06 %
Epoch 6: 9412 / 10000 , accuracy : 94.12 %
Epoch 7: 9441 / 10000 , accuracy : 94.41000000000001 %
Epoch 8: 9461 / 10000 , accuracy : 94.61 %
Epoch 9: 9475 / 10000 , accuracy : 94.75 %
Epoch 10: 9481 / 10000 , accuracy : 94.81 %
Epoch 11: 9484 / 10000 , accuracy : 94.84 %
Epoch 12: 9475 / 10000 , accuracy : 94.75 %
Epoch 13: 9495 / 10000 , accuracy : 94.95 %
Epoch 14: 9513 / 10000 , accuracy : 95.13000000000001 %
Epoch 15: 9496 / 10000 , accuracy : 94.96 %
Epoch 16: 9507 / 10000 , accuracy : 95.07 %
Epoch 17: 9499 / 10000 , accuracy : 94.99 %
Epoch 18: 9516 / 10000 , accuracy : 95.16 %
Epoch 19: 9509 / 10000 , accuracy : 95.09 %
Epoch 20: 9527 / 10000 , accuracy : 95.27 %
Epoch 21: 9526 / 10000 , accuracy : 95.26 %
Epoch 22: 9537 / 10